# Policy gradient methods

- value 기반의 RL에서는 action-value function을 통해서 간접적으로 policy를 추론해서 결정하는 형태의 학습을 진행하였다.
- 이번에는 value function을 구하지 않고 action을 선택하는 parameterized policy에 대해서 복습해보자.

- value function이 policy weights를 학습하기 위해서 사용될 수는 있지만 action을 결정하지는 않을 것이다.

- 따라서, weight parameter는 아래와 같다.<br><br>
$$ \pi(a|s, \theta) = Pr \{ A_t = a|S_t = s, \theta_t = \theta \} $$

- value function을 추가로 학습하는지에 대한 여부와는 상관없이 위 방식을 따르는 모든 methods를 **policy gradient methods**라고 한다.
- policy와 value function을 모두 학습하는 methods를 **actor-critic methods**라고 하며, actor는 학습한 policy를 의미하고, critic은 학습한 value function을 말한다.

- episodic case의 경우 objective function은,
  - parameterized policy하에 start state의 value라고 정의 가능하다.
  - \\( \eta(\theta) = v_{\pi_{\theta}}(S_0) \\)<br><br>
- Continuing task의 경우 objective function은,
  - average value를 사용하여
  - \\(\eta(\theta) = \sum_s d_{\pi_{\theta}}(s) v_{\pi_{\theta}}(s)\\)

- 위 objective function을 최대화하는 policy parameter \\(\theta\\)를 찾는 것이다. 따라서 policy RL은 optimization문제와 동일하다. optimization 문제는 gradient descent를 많이 사용한다.<br><br>
$$ \theta = \theta + \alpha \triangledown_{\theta} \eta(\theta) $$<br>
- \\(\triangledown_{\theta} \eta(\theta)\\)는 policy gradient vector이다.

## Policy Approximation and Advantages

- Policy gradient methods에서  \\(\pi(a|s, \theta)\\)가 미분가능하면 policy는 어떤 함수가 되던지 다 가능하다.
- 또한, continuous space, discrete space 모두 가능하다.

- action space가 discrete이며 큰 값이 아니라면, parameter function을 각 state-action pair에 해당하는 parameterized numerical preferences \\(h(s, a, \theta) \in \mathbb{R}\\)를 사용하는 것이다.<br><br>
- 각 state에서 가장 좋은 action은, softmax distribution의 선택 확률이 가장 높은 것이 된다.로 잡을 수 있다.

- 정의된 h는 어떤 형태로도 parameterized 가능하다.<br><br>
$$ h(s, a, \theta) = \theta^T \phi(s,a) $$

- 위에서 action을 찾는 방법을 softmax함수를 사용하면 policy approximation이 determinism에 접근한다.
- 반면 action value 기반의 \\(\epsilon-greedy\\) action 선택 방법은 \\(\epsilon\\)확률 만큼 랜덤으로 action을 선택하게 된다.
- policy approximation 방법이 action value \\(\epsilon-greedy\\)방법에 비해서 policy 추정이 간단하다.
- 이때, 가장 적절한 policy approximation은 stochastics 형태인데 action-value methods를 사용하면 stochastic policy를 구할 수 없다.

## Policy Gradient Theorem

- policy gradient 값을 구할 때 우리는 state, action, state distribution 정보를 이용한다. 하지만 policy weights을 update할 때 state와 action은 알 수 있지만 state에 대한 정보는 모두 알 수 없을 수도 있다. 그러면 gradient를 구하지 못한다.

- 따라서, policy gradient theorem이 도입되었고, gradient와 weights, value function간 관계를 규정했다.<br><br>
$$ \begin{split} \triangledown \eta(\theta) &= \sum_s d_{\pi}(s) \sum_a q_{\pi}(s, a) \triangle_{\theta} \pi(a|s, \theta) \\ &= \mathbb{E}_{\pi} \left[ \gamma^t \sum_a q_{\pi}(S_t, a) \triangledown_{\theta} \pi(a|S_t, \theta) \right] \end{split} $$

- 위 식을 action에 대해서도 고려하면,<br><br>
$$ \begin{split} \triangledown \eta(\theta) &= \mathbb{E}_\pi \left[ \gamma^t \sum_a \pi(a|S_t,\theta)q_\pi(S_t,a) \frac{\triangledown_{\theta} \pi(a|S_t,\theta)}{\pi(a|S_t,\theta)} \right] \\ &= \mathbb{E}_{\pi}\left[ \gamma^t q_{\pi}(S_t, A_t) \frac{\triangledown_{\theta} \pi(A_t|S_t,\theta)}{\pi(A_t|S_t,\theta)}\right] \\ &= \mathbb{E}_{\pi} \left[ \gamma^t G_t \frac{\triangledown_{\theta} \pi(A_t|S_t,\theta)}{\pi(A_t|S_t,\theta)} \right] \end{split} $$

- 위 식에서 value function은 어떤 것이든지 가능하고,
  - \\(\frac{\triangledown_{\theta} \pi(A_t|S_t,\theta)}{\pi(A_t|S_t,\theta)}\\)는 policy parameterization이 나타나는 항이고, likelihood ratio를 이용하여 \\(\triangledown_{\theta}log \pi(A_t|S_t, \theta)\\)로 표현가능하며 score function이라 부른다.
  - 결과적으로, linear action perference를 상요하여 softmax를 취하면, 아래의 식이 도출되고 softmax policy를 따른다고 할 수 있다.<br><br>
  
  $$ \triangledown_{\theta} log \pi(a|s, \theta) = \phi(s, a) - \sum_b \pi(b|s, \theta) \phi(s, b) $$

## REINFORCE: Monte Carlo Policy Gradient

- gradient ascent algorithm을 적용하면 아래의 식을 도출할 수 있다.<br><br>
$$ \theta_{t + 1} = \theta_t + \alpha \gamma^t G_t \frac{\triangledown_{\theta} \pi(A_t, S_t, \theta}{\pi(A_t|S_t, \theta)} $$

- REINFORCE 알고리즘은 episode가 끝나는 최종 return값을 사용하는 알고리즘이다.

## Baseline

- 임의의 \\(baseline\\)인 \\(b(s)\\)와 action value를 비교하는 형태를 사용하여 아래 식을 도출할 수 있다.<br><br>
$$ \triangledown \eta(\theta) = \sum_s d_{\pi}(s) \sum_a (q_\pi (s, a) - b(s)) \triangledown_\theta \pi(a|s, \theta) $$

- 일반적으로 baseline은 update의 예상 value를 변화시키지는 않지만 variance에 영향을 미친다. 따라서 어떤 states에서 모든 action이 모두 큰 value를 가지게 되면 작은 value를 가진 state와 차별하기 위해서 큰 baseline이 필요하다.

# Reference

[참조 블로그](https://jay.tech.blog/2017/01/03/policy-gradient-methods-part-1/)